##### Importing Packages

In [267]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sn
import numpy as np

##### Loading Data

In [10]:
TASK_PATH = r"C:\Users\jpgsa\Documents\BOLT" + "\\"
file1 = "Dataset Task 1.xlsx"
df1_raw = pd.read_excel(TASK_PATH + file1)

df1_raw.head()

,Time,Category level 0,Category level 1,Category level 2,Unique SKUs Listed,Unique SKUs Sold,# of sold SKU items,Price before Discount (includ. VAT) (EUR),Discount Value (EUR),Item COGS (net VAT) (EUR),"Waste, # of items"
0,2023-09-17,Fresh Food,🧀 Cheese,Hard Cheese,26,17,37,135.812592,10.491467,88.81738,0
1,2023-08-02,Fresh Food,🍞 Bakery,Gluten Free,1,0,0,0.000000,0.000000,0.00000,0
2,2023-08-26,Fresh Food,🥛 Dairy & Eggs,Sour Cream & Cream,14,10,26,53.454514,3.213313,31.87778,0
3,2023-09-24,Fresh Food,🥪 Fresh & Ready,Desserts & Sweets,25,13,47,61.698558,3.516517,41.22566,0
4,2023-09-15,Fresh Food,🍗 Meat & Fish,Meat & Fish Alternatives,17,3,8,13.061563,1.358066,8.68154,0


In [85]:
df1 = df1_raw.copy()
df1["perc_discount"] = df1["Discount Value (EUR)"] / df1["Price before Discount (includ. VAT) (EUR)"]
df1["category_tree"] = df1["Category level 0"] + "/" +  df1["Category level 1"] + "/" + df1["Category level 2"]
df1["avg_item_price"] = df1["Price before Discount (includ. VAT) (EUR)"] / df1["# of sold SKU items"]

##### 1) Discount Elasticity - Correlation between **qty itens** and **% discount** daily, per item

In [130]:
df_elast1 = df1.groupby('category_tree')[['perc_discount','# of sold SKU items']].corr().iloc[0::2,-1].reset_index().drop("level_1", axis = 1).rename(columns={"# of sold SKU items": "correl_disc_and_qty"})

df_temp = df1[["category_tree", "# of sold SKU items","avg_item_price"]]
df_temp = df_temp.groupby("category_tree").mean()[["# of sold SKU items","avg_item_price"]].reset_index().rename(columns={"# of sold SKU items": "avg_itens_sold_daily"})

df_elast2 = pd.merge(df_elast1, df_temp, left_on='category_tree', right_on='category_tree')
df_elast2.sort_values("correl_disc_and_qty", ascending=False, inplace=True)
df_elast2.reset_index(drop=True, inplace=True)

df_elast2.head(20)

,category_tree,correl_disc_and_qty,avg_itens_sold_daily,avg_item_price
0,Grocery Food/🥤 Beverages/More Beverages,0.935812,0.120879,3.558840
1,Fresh Food/🍗 Meat & Fish/Fresh Meat,0.503441,37.131868,4.476076
2,Fresh Food/🍗 Meat & Fish/Fresh Poultry,0.470720,40.945055,4.326510
3,Fresh Food/🥕 Fruits & Vegetables/Berries,0.454022,48.000000,2.040084
4,Grocery Food/❄️ Frozen Products/Frozen Fruits ...,0.380585,4.923077,5.278142
5,Grocery Food/🧑‍🍳 Pantry/Sugar Free Pantry,0.371779,0.252747,2.827747
6,Grocery Food/🍦 Ice Cream/Large,0.368116,69.670330,8.084641
7,Grocery Food/🥃 Spirits/Liqueurs,0.366234,1.340659,19.181716
8,Grocery Food/❄️ Frozen Products/Frozen Meals,0.341998,19.395604,3.996358
9,Grocery Food/🥃 Spirits/Vodka,0.341005,5.648352,12.427498


##### 2) Itens Afinity - Correlation of itens sold daily, between different itens

In [326]:
df_temp = df1[["Time","category_tree", "# of sold SKU items"]]
df_temp_pivot = df_temp.pivot(index='Time', columns='category_tree', values='# of sold SKU items')
corr_matrix  = df_temp_pivot.corr()
corr_matrix.rename(columns={"category_tree": "category_tree_2"})

corr_pairs = corr_matrix.unstack()
corr_pairs.index.names = ["Category A", "Category B"]
corr_pairs = corr_pairs.reset_index()
corr_pairs = corr_pairs.rename(columns={0: "Correl"})
corr_pairs = corr_pairs[corr_pairs["Category A"] != corr_pairs["Category B"]]

corr_pairs['Pair'] = np.where(corr_pairs['Category A'] > corr_pairs['Category B'] 
                              , corr_pairs['Category A'] + " + " + corr_pairs['Category B']
                              , corr_pairs['Category B'] + " + " + corr_pairs['Category A'])

corr_pairs = corr_pairs[["Pair","Correl"]]
corr_pairs.sort_values("Correl", ascending=False, inplace=True)
corr_pairs =corr_pairs.drop_duplicates(subset=["Pair"])
corr_pairs.reset_index(inplace=True, drop=True)


for index, row in corr_pairs.head(20).iterrows():
    print("Correl: ",round(row['Correl'],2), "  Combo: ",row['Pair'].split(" + "))


Correl:  0.61   Combo:  ['Grocery Food/🥤 Beverages/Kombucha', 'Fresh Food/🥕 Fruits & Vegetables/Bio & Organic']
Correl:  0.57   Combo:  ['Fresh Food/🍞 Bakery/Sliced Bread, Wraps & More', 'Fresh Food/🍗 Meat & Fish/Bacon & Cold Cuts']
Correl:  0.57   Combo:  ['Grocery Food/🍷 Wine/White Wine', 'Grocery Food/🍷 Wine/Rose Wine']
Correl:  0.56   Combo:  ['Fresh Food/🥕 Fruits & Vegetables/Vegetables', 'Fresh Food/🥕 Fruits & Vegetables/Salad Mixes']
Correl:  0.56   Combo:  ['Fresh Food/🥛 Dairy & Eggs/Yogurt & Pudding', 'Fresh Food/🍞 Bakery/Sliced Bread, Wraps & More']
Correl:  0.55   Combo:  ['Grocery Food/🍺 Beer & Cider/International Beer', 'Grocery Food/🍷 Wine/White Wine']
Correl:  0.54   Combo:  ['Grocery Food/🍺 Beer & Cider/National Favourites', 'Grocery Food/🍷 Wine/White Wine']
Correl:  0.54   Combo:  ['Grocery Food/🍬 Sweet Snacks/Biscuits', 'Fresh Food/🥪 Fresh & Ready/Desserts & Sweets']
Correl:  0.54   Combo:  ['Other/♻️ Save me!/Packaging fee', 'Grocery Food/🍦 Ice Cream/Large']
Correl: 

##### 3) Catalog Size vs. % Catalog Sold Daily

In [367]:
df_catalog = df1[["Time","category_tree","Unique SKUs Listed","Unique SKUs Sold"]].copy()
df_catalog["perc_catalog_sold"] = df_catalog["Unique SKUs Sold"] / df_catalog["Unique SKUs Listed"]

df_catalog_grouped = df_catalog[df_catalog.columns[1:]].groupby("category_tree").mean().reset_index()
df_catalog_grouped = df_catalog_grouped.sort_values("perc_catalog_sold", ascending=True)
df_catalog_grouped[df_catalog_grouped["Unique SKUs Listed"] > 20].head(20)

,category_tree,Unique SKUs Listed,Unique SKUs Sold,perc_catalog_sold
149,Grocery Non-Food/🎲 Toys & Games/Games,34.483516,0.450549,0.013085
150,Grocery Non-Food/🎲 Toys & Games/Toys,68.406593,1.164835,0.017020
155,Grocery Non-Food/🏡 Home Accessories/Stationery,55.274725,1.945055,0.035245
153,Grocery Non-Food/🏡 Home Accessories/Electronics,35.934066,1.835165,0.051029
84,Grocery Food/🍷 Wine/Red Wine,90.164835,7.813187,0.087678
161,Grocery Non-Food/👶 Baby Care/Baby Food,28.241758,2.637363,0.093556
190,Not defined/Not defined/Not defined,33.681319,3.703297,0.109570
154,Grocery Non-Food/🏡 Home Accessories/More Home ...,23.340659,2.923077,0.125159
186,Grocery Non-Food/🩹 Health & Safety/Feel Better,25.000000,3.252747,0.130110
54,"Grocery Food/☕️ Coffee, Tea & Cocoa/Tea",38.395604,5.285714,0.140851


##### 4) Estimated Total Loss (based on wasted items and avg COGS)

In [339]:
df_waste = df1[["category_tree","# of sold SKU items","Item COGS (net VAT) (EUR)","Waste, # of items"]]
df_waste_grouped = df_waste.groupby("category_tree").sum().reset_index()

df_waste_grouped["Avg. COGS per Item"] = df_waste_grouped["Item COGS (net VAT) (EUR)"] / df_waste_grouped["# of sold SKU items"]
df_waste_grouped["Est. Total Loss (EUR)"] = df_waste_grouped["Waste, # of items"] * df_waste_grouped["Avg. COGS per Item"]

df_waste_grouped = df_waste_grouped.sort_values("Est. Total Loss (EUR)", ascending=False)
df_waste_grouped.head(20)

,category_tree,# of sold SKU items,Item COGS (net VAT) (EUR),"Waste, # of items",Avg. COGS per Item,Est. Total Loss (EUR)
15,Fresh Food/🥕 Fruits & Vegetables/Fruits,24094,26588.27060,2779,1.103522,3066.688968
2,Fresh Food/🍗 Meat & Fish/Fresh Meat,3379,9653.82245,897,2.857006,2562.734163
18,Fresh Food/🥕 Fruits & Vegetables/Vegetables,22087,15264.95601,1997,0.691129,1380.183690
5,Fresh Food/🍗 Meat & Fish/Ready To Cook,413,1049.36181,408,2.540828,1036.657672
0,Fresh Food/🍗 Meat & Fish/Bacon & Cold Cuts,11459,16428.35767,716,1.433664,1026.503542
17,Fresh Food/🥕 Fruits & Vegetables/Salad Mixes,4558,3275.93933,1412,0.718723,1014.836844
6,Fresh Food/🍗 Meat & Fish/Sausages,2883,5104.95021,551,1.770708,975.659926
26,Fresh Food/🥛 Dairy & Eggs/Yogurt & Pudding,18887,14980.25385,1201,0.793152,952.575045
28,Fresh Food/🥪 Fresh & Ready/Desserts & Sweets,3471,4534.18263,702,1.306304,917.025700
13,Fresh Food/🥕 Fruits & Vegetables/Berries,4368,4789.56322,808,1.096512,885.981475


##### 5) Net Revenue and % Margin